# <font color ="#0B6FA6">Capítulo da Sociedade de Inteligência Computacional do IEEE UFPB </font>

## Preencha seus dados:

In [ ]:
#@title Preencha os campos abaixo:

nome = 'Laura de Faria' #@param {type:"string"}


## Análise de Dados
---

**Descrição:**
\
Nessa seção, você adentrará no âmbito da análise de dados, que se trata do processo de limpeza, inspeção, modelagem e transformação de dados, corroborando para uma melhor interpretação dos dados e tomadas de decisão mais incisivas. Após essa breve introdução, você terá de analisar o dataset [Ocorrências Aviação](https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/ocorrencias_aviacao.csv). O dataset é formado pelos seguintes dados:


### 1 - Leitura dos dados e realização de descrição estatística
---

A seguir, o dataframe já estará montado pelo método `pd.read_csv()` da biblioteca `pandas`, a partir da execução da célula.

Dica: Você pode acessar a documentação da biblioteca `pandas` [clicando aqui](https://pandas.pydata.org/docs/getting_started/index.html#getting-started). Após o processamento do dataframe, você terá de realizar uma descrição estatística dos dados.

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/ocorrencias_aviacao.csv', sep = ',')

# Dimens�o do dataset (Linhas x Colunas)
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### 2 - Limpeza dos dados

---

Após verificar o dataset acima e a relação das descrições com os valores existentes, realize o processo de limpeza dos dados, verificando possíveis inconsistências entre as instâncias (linhas que compõem o dataframe).

In [ ]:
# Verificar dados faltantes por coluna
missing_data = df.isnull().sum()
print(missing_data)

In [ ]:
df = df.drop(columns=['investigacao_aeronave_liberada', 'divulgacao_dia_publicacao'])

In [ ]:
# Verificar duplicatas
duplicatas = df.duplicated()

total_duplicatas = duplicatas.sum()

print(f"N�mero de duplicatas encontradas: {total_duplicatas}")

In [ ]:
df['ocorrencia_dia'] = pd.to_datetime(df['ocorrencia_dia'], format='%Y-%m-%d')

In [ ]:
df['ocorrencia_horario'] = pd.to_datetime(df['ocorrencia_horario'], format='%H:%M:%S', errors='coerce')

In [ ]:
df.info()

### 3 - Feature Engineering e Dados Categóricos
---

3.1 - Em um problema de _Data Science_, é comum nos depararmos com um dataset cujos atributos não esclareçam com efetividade o comportamento de um dado fenômeno. Desta forma, cientistas de dados devem estar aptos a manipular atributos existentes e assim definir novos atributos, de acordo com os critérios estabelecidos. Este processo é descrito como _Feature Engineering_, e trata apenas da manipulação algébrica e/ou lógica de atributos (colunas, no contexto prático) com o intuito de alimentar o seu modelo de Machine Learning com dados relevantes à resolução do problema.
Nesta etapa, você deverá utilizar sua intuição aliando criatividade para decidir como novos atributos podem ser definidos, a partir dos dados que você já possui.

In [ ]:
# Agrupando ocorr�ncias por regi�es

regioes = {
    'N': ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    'NE': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    'CO': ['DF', 'GO', 'MT', 'MS'],
    'SE': ['ES', 'MG', 'RJ', 'SP'],
    'S': ['PR', 'RS', 'SC']
}

def agrupa_regiao(uf):
    for regiao, estados in regioes.items():
        if uf in estados:
            return regiao
    return 'Desconhecido'

df['ocorrencia_regiao'] = df['ocorrencia_uf'].apply(agrupa_regiao)

In [ ]:
# Contagem de ocorr�ncias por regi�o

df['ocorrencias_por_regiao'] = df.groupby('ocorrencia_regiao')['ocorrencia_regiao'].transform('count')

In [ ]:
# Flag para gravidade da ocorr�ncia

tipos_graves = ['ACIDENTE', 'INCIDENTE GRAVE']
df['ocorrencia_grave'] = df['ocorrencia_classificacao'].apply(lambda x: 1 if x in tipos_graves else 0)

In [ ]:
# Ano da ocorr�ncia

df['ocorrencia_ano'] = df['ocorrencia_dia'].dt.year

3.2 - Algoritmos de Machine Learning só funcionam com valores numéricos. Isto significa que antes de treinar qualquer modelo, deve-se transformar os dados categóricos em dados numéricos. Encontre uma forma para que seu modelo consiga interpretar todos os atributos não-numéricos como numéricos.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Agrupando as colunas categ�ricas
categorical_columns = [
    'ocorrencia_classificacao', 'ocorrencia_tipo', 'ocorrencia_tipo_categoria', 'ocorrencia_tipo_icao',
    'ocorrencia_cidade', 'ocorrencia_uf', 'ocorrencia_pais', 'ocorrencia_aerodromo',
    'investigacao_status', 'divulgacao_relatorio_numero', 'divulgacao_relatorio_publicado',
    'ocorrencia_saida_pista', 'ocorrencia_dia_extracao', 'ocorrencia_regiao'
]

# Aplicando a técnica One-Hot Encoding
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

print(df.head())
print(df.info())

### 4 - Balanceamento de dados



*(SE NECESSÁRIO UTILIZAR, JUSTIFIQUE)*

O balanceamento de classes é uma técnica de pré-processamento usada em problemas de classificação quando as classes alvo estão desproporcionalmente representadas nos dados. É necessário quando essa distribuição desigual prejudica o desempenho do modelo. Pode ser feito aumentando a classe minoritária (oversampling) ou reduzindo a classe majoritária (undersampling). A escolha depende do contexto e das características dos dados. É crucial para garantir que o modelo seja treinado de maneira equilibrada para todas as classes, melhorando assim sua capacidade de generalização e precisão nas previsões.

In [ ]:
import matplotlib.pyplot as plt

df.hist(figsize=(15,15))
plt.show()

### 5 - Seleção de características
---

Se o conjunto de dados contiver muitos atributos, é realizada a seleção de características para reduzir a dimensionalidade e melhorar a eficiência e precisão dos modelos. Isso pode ser feito usando técnicas como análise de correlação, análise de componentes principais (PCA) ou algoritmos de seleção de características.

In [ ]:
# Análise de correlação

import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = df.corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correla��o')
plt.show()

# Caracter�sticas com correla��o com 'ocorrencia_grave'
target_corr = abs(corr_matrix['ocorrencia_grave']).sort_values(ascending=False)
relevant_features = target_corr.index.tolist()

print("Caracter�sticas relevantes ordenadas por correla��o com 'ocorrencia_grave':")
print(target_corr)

Os resultados da correlação indicam quais características estão mais fortemente associadas à variável 'ocorrencia_grave' no conjunto de dados.

- Primeiramente, observamos que tanto a classificação da ocorrência como INCIDENTE GRAVE quanto a própria variável alvo 'ocorrencia_grave' apresentam uma correlação positiva perfeita de 1. Isso significa que o fato de uma ocorrência ser classificada como INCIDENTE GRAVE está diretamente relacionado à sua gravidade.
- Além disso, características como o número do relatório de divulgação, que possui uma correlação de 0.872415, e a informação de se o relatório foi publicado, com uma correlação de 0.607092, mostram fortes associações positivas com ocorrências graves.
- Por outro lado, características como o aeródromo específico e cidades como ARAGUAÍNA mostram correlações muito baixas, próximas de zero, com 'ocorrencia_grave', sugerindo que essas variáveis têm pouca influência na determinação da gravidade das ocorrências.

Por fim, temos que essas correlações nos ajudam a identificar quais características podem ser mais relevantes para prever ou entender ocorrências graves no conjunto de dados.

In [ ]:
# Convers�o de coluna de data para ano
df['ano_ocorrencia'] = df['ocorrencia_dia'].dt.year

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X = df.drop(['codigo_ocorrencia', 'ocorrencia_dia', 'ocorrencia_horario'], axis=1)
y = df['ocorrencia_grave']  # Vari�vel alvo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando o Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))

Os resultados apresentados com o Random Forest indicam que o modelo atingiu uma performance muito alta:

- Precision e Recall: Ambas as classes (0 e 1) têm precision e recall de 1.00, o que significa que o modelo não teve falsos positivos nem falsos negativos para nenhuma das classes. Isso indica que o modelo está previsando com perfeição ambas as classes.
- F1-score: O F1-score também é 1.00 para ambas as classes, o que confirma a excelente precisão e recall observados. O valor alto de F1-score indica que o modelo é robusto e tem um bom equilíbrio entre precision e recall.
- Accuracy: A acurácia do modelo é 1.00, o que significa que todos os exemplos foram classificados corretamente. Isso é um indicativo de que o modelo está performando muito bem na classificação dos dados.

### 6 - Análise Exploratória dos Dados
---

Antes de qualquer análise aprofundada de um conjunto de dados, é crucial entender como esses dados estão distribuídos, quais valores são mais comuns, a dispersão dos dados e se há valores que se destacam do restante do conjunto. Essas informações são fundamentais para a análise exploratória de dados e são conhecidas como estatísticas descritivas. Assim, realize uma análise exploratória sobre os dados deste problema, utilizando a biblioteca `matplotlib.pyplot` ou `seaborn` para criar gráficos que representem estas medidas. Utilize boxplots para visualizar dados muito discrepantes entre as amostras do dataset.

**Perguntas para responder:**

- Quais são as principais causas de ocorrência e, eventualmente, queda de aviões?
- Como isso progrediu nos últimos anos? Houve menos ou mais ocorrências?
- Quais estados registraram mais ocorrências?
- Há mais ocorrências finalizadas ou ativas?

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/ocorrencias_aviacao.csv', sep = ',')

In [ ]:
# Contagem de ocorr�ncias por estado
top_estados = df['ocorrencia_uf'].value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_estados.values, y=top_estados.index, hue=top_estados.index, palette='pink', dodge=False)
plt.title('Estados com Maior N�mero de Ocorr�ncias de Avia��o')
plt.xlabel('N�mero de Ocorr�ncias')
plt.ylabel('Estado')
plt.show()

In [ ]:
# Contagem de ocorr�ncias por ano
df['ocorrencia_dia'] = pd.to_datetime(df['ocorrencia_dia'], format='%Y-%m-%d')
df['ocorrencia_ano'] = df['ocorrencia_dia'].dt.year
ocorrencias_por_ano = df['ocorrencia_ano'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.plot(ocorrencias_por_ano.index, ocorrencias_por_ano.values, marker='o', linestyle='-', color='pink')
plt.title('Progress�o de ocorr�ncias de avia��o ao longo dos anos')
plt.xlabel('Ano')
plt.ylabel('N�mero de ocorr�ncias')
plt.grid(True)
plt.show()

In [ ]:
# Contagem das principais causas de ocorr�ncia
top_causas = df['ocorrencia_tipo'].value_counts().head(15)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_causas.values, y=top_causas.index, palette='mako', saturation=0.8)
plt.title('Principais causas de ocorr�ncias de avia��o')
plt.xlabel('N�mero de ocorr�ncias')
plt.ylabel('Causa da ocorr�ncia')
plt.show()

In [ ]:
# Contagem de ocorr�ncias por status de investiga��o
status_investigacao = df['investigacao_status'].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=status_investigacao.values, y=status_investigacao.index, palette='husl')
plt.title('Distribui��o de ocorr�ncias por status de investiga��o')
plt.xlabel('N�mero de ocorr�ncias')
plt.ylabel('Status da investiga��o')
plt.show()

In [ ]:
plt.figure(figsize=(8, 16))
sns.boxplot(x='investigacao_status', y='ocorrencia_tipo', data=df, palette='husl')
plt.title('Distribui��o de ocorr�ncias por status de investiga��o')
plt.xlabel('Status de investiga��o')
plt.ylabel('N�mero de ocorr�ncias')
plt.show()

- As principais causas de ocorrência são, respectivamente, falha do motor em voo, estouro de pneu e falha ou mal funcionamento de sistema/componente

- Nos últimos anos, o progresso dessas ocorrências teve um aumento até 2012, em que houve um pico de acidentes. Entre 2012 e 2013 não houve mudança significativa mas, entre 2013 e 2014, houve um declínio de ocorrências, com uma diminuição constante até 2016, onde aumentou levemente, mas voltou a cair até 2018, que foi o último ano estudado.

- Os estados que mais registraram ocorrências foram, em ordem:
  1. SP
  2. RJ
  3. MG
  4. PR
  5. RS

- Há significativamente mais ocorrências finalizadas do que ativas. Também foi comprovado que não há ocorrências reabertas.